# Congressional Votes
- Pull Voting data by House Rep
- Join with Bill data on Bill ID
- Understand relationship between Bill sponsorship with party lines, ...

In [8]:
from urllib.request import urlopen
import bs4
import requests
from lxml import html
from lxml.cssselect import CSSSelector
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# House Vote Data

In [180]:
url = 'http://clerk.house.gov/evs/2017/roll699.xml'
source = requests.get(url)
ntree = html.document_fromstring(source.content)

In [181]:
party = ntree.xpath('//party/text()')
yea = ntree.xpath('//yea-total/text()')
nay = ntree.xpath('//nay-total/text()')
present = ntree.xpath('//present-total/text()')
not_voting = ntree.xpath('//not-voting-total/text()')

party = [*party, 'Total']

In [182]:
congress = ntree.xpath('//congress/text()')
rollnum = ntree.xpath('//rollcall-num/text()')

congress = [*congress, *congress, *congress, *congress]
rollnum = [*rollnum, *rollnum, *rollnum, *rollnum]

In [183]:
print(rollnum)

['699', '699', '699', '699']


In [184]:
votingdf = pd.DataFrame({'congress':congress, 'rollnum':rollnum, 'party':party, 'yea':yea, 'nay':nay, 'present':present, 'not_voting':not_voting})
votingdf = votingdf[['congress', 'rollnum', 'party', 'yea', 'nay', 'present', 'not_voting']]

In [185]:
votingdf

,congress,rollnum,party,yea,nay,present,not_voting
0,115,699,Republican,224,12,0,3
1,115,699,Democratic,0,189,0,4
2,115,699,Independent,0,0,0,0
3,115,699,Total,224,201,0,7


In [174]:
votingdf_flat = pd.melt(votingdf, id_vars=['congress','rollnum','party'], 
                        value_vars=['yea','nay','present','not_voting'], 
                        var_name='vote_type', value_name='vote_count')

In [ ]:
votingdf_flat

In [186]:
votingdf_699 = votingdf

In [179]:
votingdf_692

,congress,rollnum,party,yea,nay,present,not_voting
0,115,692,Republican,227,12,0,0
1,115,692,Democratic,0,191,0,2
2,115,692,Independent,0,0,0,0
3,115,692,Total,227,203,0,2


In [187]:
votingdf_699

,congress,rollnum,party,yea,nay,present,not_voting
0,115,699,Republican,224,12,0,3
1,115,699,Democratic,0,189,0,4
2,115,699,Independent,0,0,0,0
3,115,699,Total,224,201,0,7


# House Bill Data

In [91]:
bill_url = 'https://www.govinfo.gov/bulkdata/BILLSTATUS/115/hr/BILLSTATUS-115hr1.xml'
bill_source = requests.get(bill_url)
bill_ntree = html.document_fromstring(bill_source.content)

In [155]:
bill_rollnum = bill_ntree.xpath('//recordedvote/rollnumber/text()')
bill_chamber = bill_ntree.xpath('//recordedvote/chamber/text()')
bill_action = bill_ntree.xpath('//recordedvote/fullactionname/text()')
bill_date = bill_ntree.xpath('//recordedvote/date/text()')
bill_voteurl = bill_ntree.xpath('//recordedvote/url/text()')

In [156]:
votingdf = pd.DataFrame({'bill_rollnum':bill_rollnum, 'bill_chamber':bill_chamber, 'bill_action':bill_action, 
                         'bill_date':bill_date, 'bill_voteurl':bill_voteurl})

In [157]:
col_seq = ['bill_date','bill_rollnum','bill_chamber','bill_action','bill_voteurl']
votingdf = votingdf.reindex(columns=col_seq)

In [158]:
votingdf

,bill_date,bill_rollnum,bill_chamber,bill_action,bill_voteurl
0,2017-12-20T17:56:09Z,699,House,Vote on House Agreeing to the Senate Action,http://clerk.house.gov/evs/2017/roll699.xml
1,2017-12-20T05:33:45Z,323,Senate,FREE TEXT,http://www.senate.gov/legislative/LIS/roll_cal...
2,2017-12-20T05:13:26Z,322,Senate,MOTION TO WAIVE (AMENDMENT),http://www.senate.gov/legislative/LIS/roll_cal...
3,2017-12-19T20:46:01Z,321,Senate,MOTION TO CONSIDER,http://www.senate.gov/legislative/LIS/roll_cal...
4,2017-12-19T19:28:52Z,692,House,Placeholder Text for H42510,http://clerk.house.gov/evs/2017/roll692.xml
5,2017-12-19T19:20:59Z,691,House,Placeholder Text for H42411,http://clerk.house.gov/evs/2017/roll691.xml
6,2017-12-06T22:51:38Z,309,Senate,MOTION TO INSTRUCT,http://www.senate.gov/legislative/LIS/roll_cal...
7,2017-12-06T21:30:12Z,308,Senate,MOTION TO INSTRUCT,http://www.senate.gov/legislative/LIS/roll_cal...
8,2017-12-06T21:02:34Z,307,Senate,MOTION TO INSTRUCT,http://www.senate.gov/legislative/LIS/roll_cal...
9,2017-12-06T20:22:15Z,306,Senate,FREE TEXT,http://www.senate.gov/legislative/LIS/roll_cal...
